In [26]:
cd /SNS/users/lj7/dv/sns-chops/resolution/ARCS/

/SNS/users/lj7/dv/sns-chops/resolution/ARCS


In [1]:
#Program to load Vanadium or empty Can powder files and perform a constant-Q cut along the middle Q to look at energy resolution and intensity.
import matplotlib.pyplot as plt
from mantid import plots
from mantid.simpleapi import Load, ConvertToMD, BinMD, ConvertUnits, Rebin
from matplotlib.colors import LogNorm
import numpy as np
import mantid.simpleapi as msa
from mantid import mtd

In [2]:
%matplotlib notebook

In [5]:
RunNumbers=range(107476,107648)+range(108331,108457) # FC1
# quick test: 
# RunNumbers=range(107476,107480)
# runno = 107623
# RunNumbers=range(runno,runno+1)

print(RunNumbers)
IPTS=21387
datadir="/SNS/ARCS/IPTS-"+str(IPTS)+"/shared/autoreduce/"

[107476, 107477, 107478, 107479, 107480, 107481, 107482, 107483, 107484, 107485, 107486, 107487, 107488, 107489, 107490, 107491, 107492, 107493, 107494, 107495, 107496, 107497, 107498, 107499, 107500, 107501, 107502, 107503, 107504, 107505, 107506, 107507, 107508, 107509, 107510, 107511, 107512, 107513, 107514, 107515, 107516, 107517, 107518, 107519, 107520, 107521, 107522, 107523, 107524, 107525, 107526, 107527, 107528, 107529, 107530, 107531, 107532, 107533, 107534, 107535, 107536, 107537, 107538, 107539, 107540, 107541, 107542, 107543, 107544, 107545, 107546, 107547, 107548, 107549, 107550, 107551, 107552, 107553, 107554, 107555, 107556, 107557, 107558, 107559, 107560, 107561, 107562, 107563, 107564, 107565, 107566, 107567, 107568, 107569, 107570, 107571, 107572, 107573, 107574, 107575, 107576, 107577, 107578, 107579, 107580, 107581, 107582, 107583, 107584, 107585, 107586, 107587, 107588, 107589, 107590, 107591, 107592, 107593, 107594, 107595, 107596, 107597, 107598, 107599, 107600,

# gather data

In [6]:
PlotTag=0

data_table = {}

for RunNumber in RunNumbers:
    print (RunNumber)
    try:
        w=msa.CreateSingleValuedWorkspace()
        #LoadNexusLogs(w,"/SNS/ARCS/IPTS-"+str(IPTS)+"/data/ARCS_"+str(RunNumber)+'_event.nxs')
        msa.LoadNexusLogs(w,"/SNS/ARCS/IPTS-"+str(IPTS)+"/nexus/ARCS_"+str(RunNumber)+'.nxs.h5')
        RunParams=w.getRun()
        Energy=RunParams["BL18:Chop:Skf0:EnergyUserReq"].getStatistics().mean
        #print Energy
        Chopperpos=RunParams["chtrans"].getStatistics().mean
        #print Chopperpos
        Chopper1=RunParams["Speed1"].getStatistics().mean
        Chopper2=RunParams["Speed2"].getStatistics().mean
        Chopper3=RunParams["Speed3"].getStatistics().mean
        if Chopperpos>400: 
            Chopper=2 
        elif Chopperpos<1: 
            Chopper=1 
        else: 
            Chopper=0

        
        # generate a nice 2D multi-dimensional workspace
        data = msa.LoadNXSPE(datadir+'ARCS_'+str(RunNumber)+'_autoreduced.nxspe')
        values=msa.ConvertToMDMinMaxLocal('data',QDimensions='|Q|', dEAnalysisMode='Direct')
        minQ,minE=values.MinValues
        maxQ,maxE=values.MaxValues
        
        md = msa.ConvertToMD(InputWorkspace=data, QDimensions='|Q|', dEAnalysisMode='Direct')
        sqw = msa.BinMD(InputWorkspace=md,
                    AlignedDim0='|Q|,'+str(minQ)+','+str(maxQ)+',100',
                    AlignedDim1='DeltaE,'+ str(minE) +',' +str(maxE*0.8) +',100')

        #2D plot
        if PlotTag==1:
            fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
            c = ax.pcolormesh(sqw, vmin=0., vmax=0.5e-3)
            cbar=fig.colorbar(c)
            cbar.set_label('Intensity (arb. units)') #add text to colorbar
            ax.set_title('Run '+str(RunNumber)+',Ei='+str(Energy)+'meVChoppers=['+str(Chopper1)+','+str(Chopper2)+','+str(Chopper3)+']')
            fig.show()


        # generate a 1D multi-dimensional workspace


        sqw_line= msa.BinMD(
            InputWorkspace=md,
            #AlignedDim0='|Q|,' +str((minQ+maxQ)/3-0.01*maxQ) +','+ str((minQ+maxQ)/3+0.01*maxQ) +',1',
            AlignedDim0='|Q|,%s,%s,1' % ((minQ*2+maxQ)/3 , (minQ+2*maxQ)/3),
            AlignedDim1='DeltaE,'+ str(minE) +',' +str(maxE*0.8) +',100')

        sqw_line_Hist=msa.ConvertMDHistoToMatrixWorkspace('sqw_line', Normalization='NumEventsNormalization')

        # plots 1D multi-dimensional workspace
        if PlotTag==1:
            fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
            c = ax.errorbar(sqw_line_Hist,label='Data')
            ax.legend()
            ax.set_title('Run '+str(RunNumber)+',Ei='+str(Energy)+'meV,|Q|=[' +str((minQ+maxQ)/3-0.01*maxQ) +','+ str((minQ+maxQ)/3+0.01*maxQ) +']' )
            fig.show()
        xbb, y, Err = sqw_line_Hist.extractX(), sqw_line_Hist.extractY(), sqw_line_Hist.extractE()
        xbb.shape = y.shape = Err.shape = -1,
        x = (xbb[1:]+xbb[:-1])/2
        IE = x,y,Err

        RunParams2=data.getRun()
        Ei=RunParams2["Ei"].value
        Q=(minQ+maxQ)/3
        # array=[0,RunNumber, Energy, Ei, Chopper, round(Chopper1), round(Chopper2), round(Chopper3), Height, dHeight, Center, dCenter, Sigma, dSigma, Q]
        print("Run=",RunNumber,", Energy=",Energy,"meV, Chopper=",Chopper,"ChopperPosition=",Chopperpos)
        #print array
        data_table[RunNumber] = [
            Energy, Ei, 
            Chopper, round(Chopper1), round(Chopper2), round(Chopper3),
            IE,
            Q]
        
    except:
        raise
        print(RunNumber,": Errors, either file not in directory or some syntax, or fitting / plot errors")


107476
('Run=', 107476, ', Energy=', 8.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107477
('Run=', 107477, ', Energy=', 10.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107478
('Run=', 107478, ', Energy=', 12.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107479
('Run=', 107479, ', Energy=', 15.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107480
('Run=', 107480, ', Energy=', 20.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107481
('Run=', 107481, ', Energy=', 30.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107482
('Run=', 107482, ', Energy=', 40.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107483
('Run=', 107483, ', Energy=', 50.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107484
('Run=', 107484, ', Energy=', 60.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107485
('Run=', 107485, ', Energy=', 70.0, 'meV, Chopper=', 1, 'ChopperPos

('Run=', 107555, ', Energy=', 1000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107556
('Run=', 107556, ', Energy=', 1200.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107557
('Run=', 107557, ', Energy=', 1500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107558
('Run=', 107558, ', Energy=', 1800.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107559
('Run=', 107559, ', Energy=', 2000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107560
('Run=', 107560, ', Energy=', 2500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107561
('Run=', 107561, ', Energy=', 3000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107562
('Run=', 107562, ', Energy=', 30.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107563
('Run=', 107563, ', Energy=', 40.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107564
('Run=', 107564, ', Energy=', 50.0, 'meV, Chopper=', 1, 'Ch

('Run=', 107634, ', Energy=', 900.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107635
('Run=', 107635, ', Energy=', 1000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107636
('Run=', 107636, ', Energy=', 1200.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107637
('Run=', 107637, ', Energy=', 1500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107638
('Run=', 107638, ', Energy=', 1800.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107639
('Run=', 107639, ', Energy=', 2000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107640
('Run=', 107640, ', Energy=', 2500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107641
('Run=', 107641, ', Energy=', 3000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107642
('Run=', 107642, ', Energy=', 3500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6710330000000001)
107643
('Run=', 107643, ', Energy=', 4000.0, 'meV, Chopper=', 1

('Run=', 108396, ', Energy=', 500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108397
('Run=', 108397, ', Energy=', 600.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108398
('Run=', 108398, ', Energy=', 700.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108399
('Run=', 108399, ', Energy=', 800.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108400
('Run=', 108400, ', Energy=', 900.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108401
('Run=', 108401, ', Energy=', 1000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108402
('Run=', 108402, ', Energy=', 1500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108403
('Run=', 108403, ', Energy=', 1800.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108404
('Run=', 108404, ', Energy=', 2000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.6698935100000001)
108405
('Run=', 108405, ', Energy=', 2500.0, 'meV, Chopper=', 1, 'C

# Directly get FWHM using interpolation

In [9]:
import scipy

In [30]:
%%time
plot = False
rows = []
for RunNumber, record in data_table.items():
    Energy, Ei, \
        Chopper, Chopper1, Chopper2, Chopper3,\
        IE,\
        Q = record
    x,y,Err = IE
    x_interp = np.arange(-Ei/2, Ei/2, Ei/1000)
    # y_interp = np.interp(x_interp, x,y)
    y_interp2 = scipy.interpolate.interp1d(x,y, kind='quadratic', bounds_error=False)(x_interp)

    # print Ei, Energy
    if plot:
        plt.figure()
        plt.plot(x, y, '+')
        plt.plot(x_interp, y_interp2, 'o')
    ymax = np.nanmax(y_interp2)
    hm = ymax/2
    middle = x_interp[y_interp2>hm]
    if middle.size:
        FWHM = middle[-1]-middle[0]
    else:
        FWHM = -1.
    Height = ymax
    Sigma = FWHM/2.355
    row=[
        0,RunNumber, 
        Energy, Ei, 
        Chopper, Chopper1, Chopper2, Chopper3,
        Height, FWHM, Sigma, Q
    ]
    rows.append(row)
    continue

CPU times: user 129 ms, sys: 2.91 ms, total: 132 ms
Wall time: 131 ms


/SNS/software/miniconda2/envs/mcvine-unstable/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in greater
/SNS/software/miniconda2/envs/mcvine-unstable/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: All-NaN slice encountered


In [31]:
np.savetxt(
    "./V_Cali_Int_Res_FC1_2018_v2.dat",
    rows,
    header='RunNumber Energy Ei Chopper Chopper1 Chopper2 Chopper3 Height FWHM Sigma Q'
)